In [ ]:
import pathlib
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential


In [ ]:
# dataset path
pic_path = pathlib.Path(r'C:\Users\DHINOOP\Downloads\dataset')




In [ ]:
# check folders
list(pic_path.iterdir())

In [ ]:
import pathlib

pic_path = pathlib.Path(r"C:\Users\DHINOOP\Downloads\dataset")

mammootty_list = list(pic_path.glob("mammooty/**/*"))
mohanlal_list = list(pic_path.glob("mohanlal/**/*"))

print(f"Mammootty images: {len(mammootty_list)}")
print(f"Mohanlal images: {len(mohanlal_list)}")

if len(mammootty_list) > 0:
    print("Sample Mammootty:", mammootty_list[0])
else:
    print("⚠ No Mammootty images found.")

if len(mohanlal_list) > 0:
    print("Sample Mohanlal:", mohanlal_list[0])
else:
    print("⚠ No Mohanlal images found.")


In [ ]:
mammootty_list = list(pic_path.glob("mammooty/**/*"))
mohanlal_list = list(pic_path.glob("mohanlal/**/*"))

pic_dict = {'mammootty': mammootty_list, 'mohanlal': mohanlal_list}
pic_class = {'mammootty': 0, 'mohanlal': 1}

# Optional: print some paths
for i in pic_dict:
    print(f"\n{i}:")
    for j in pic_dict[i][:3]:  # print first 3 images from each class
        print(j)


In [ ]:
x, y = [], []

for label in pic_dict:
    for img_path in pic_dict[label]:
        img = cv2.imread(str(img_path))

        if img is None:
            print("Skipping unreadable image:", img_path)
            continue

        img = cv2.resize(img, (100, 100))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0

        x.append(img)
        y.append(pic_class[label])

x = np.array(x)
y = np.array(y)

print(x.shape, y.shape)


In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, train_size=0.8, random_state=1)


In [ ]:
model = Sequential([
    Conv2D(50, (3,3), activation='relu', input_shape=(100,100,3), strides=1, padding='valid'),
    MaxPool2D(pool_size=(2,3)),
    Conv2D(100, (3,3), activation='relu', strides=1, padding='valid'),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(), 
              loss=SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

history = model.fit(xtrain, ytrain, 
                    batch_size=32, epochs=100, 
                    validation_data=(xtest, ytest))

# save the model
model.save('face.h5')


In [ ]:
# Load model
model = tf.keras.models.load_model('face.h5')

# Labels
labels = {0: 'Mammootty', 1: 'Mohanlal'}


In [ ]:
def detect_faces(frame, model, labels):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        cv2.putText(frame, 'No face detected', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (0,0,255), 2, cv2.LINE_AA)
        return frame

    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_resized = cv2.resize(face_rgb, (100,100))
        face_resized = np.expand_dims(face_resized/255.0, axis=0)
        
        pred = model.predict(face_resized)
        label = labels[pred.argmax()]
        
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.9, (0,255,0), 2)
    return frame


In [ ]:
test_image_path = r'C:\Users\DHINOOP\Downloads\lal.jpg'  # change this to your test image
frame = cv2.imread(test_image_path)

if frame is None:
    print("Error: Unable to load image.")
else:
    output_image = detect_faces(frame, model, labels)
    cv2.imshow("Prediction", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
test_image_path = r'C:\Users\DHINOOP\Downloads\ikka.web'  # change this to your test image
frame = cv2.imread(test_image_path)

if frame is None:
    print("Error: Unable to load image.")
else:
    output_image = detect_faces(frame, model, labels)
    cv2.imshow("Prediction", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:

import cv2
import numpy as np
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model("face.h5")

# Labels mapping
labels = {0: 'Mammootty', 1: 'Mohanlal'}

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Prediction function
def predict_face(face_img):
    face_img = cv2.resize(face_img, (100, 100))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = model.predict(face_img)
    return labels[np.argmax(preds)]

# Open webcam
cap = cv2.VideoCapture(0)  # use 0 for the default camera

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        label = predict_face(face_img)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow("Live Face Recognition", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()
